<a href="https://colab.research.google.com/github/code-with-ayesha/chatbots/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio langchain langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Send the student's question to Groq's API and get a response
        response = llm.invoke(student_input)
        return response.strip()  # Clean the response
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any question, and I'll try my best to help you!"  # Description of the app
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2fefd6dacb9f2c3103.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Send the student's question to Groq's API and get a response
        response = llm.invoke(student_input)

        # Check if the response is an 'AIMessage' object and extract text
        if hasattr(response, 'text'):
            return response.text.strip()  # Extract text from the response object
        else:
            return "No valid response received."

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any question, and I'll try my best to help you!"  # Description of the app
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e81fb0685eea602252.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Send the student's question to Groq's API and get a response
        response = llm.invoke(student_input)

        # Print the raw response for inspection
        print("Raw response:", response)

        # Check if the response has a 'text' attribute
        if hasattr(response, 'text'):
            return response.text.strip()  # Extract text from the response object
        else:
            return f"No valid 'text' attribute found. Full response: {response}"

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any question, and I'll try my best to help you!"  # Description of the app
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06c279b498f08d14cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Send the student's question to Groq's API and get a response
        response = llm.invoke(student_input)

        # Extract the content from the response
        content = response.get('content', 'No content found.')
        return content.strip()  # Return the content text

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any question, and I'll try my best to help you!"  # Description of the app
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e60bd85e93ed72c910.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Send the student's question to Groq's API and get a response
        response = llm.invoke(student_input)

        # Access the content directly from the response
        content = response.content  # Accessing content directly
        return content.strip()  # Return the content text

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any question, and I'll try my best to help you!"  # Description of the app
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aec568490203df9885.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Modify the prompt to be more educational
        prompt = f"You are a helpful assistant for students. Answer the following question in a way that helps a student understand the topic better:\n{student_input}"

        # Send the student's question to Groq's API and get a response
        response = llm.invoke(prompt)

        # Access the content directly from the response
        content = response.content  # Accessing content directly
        return content.strip()  # Return the content text

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any educational question, and I'll help you learn!"  # Student-focused description
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa8ab957a823ee70ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata

# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to check if the query is student-related
def is_student_query(query):
    # List of keywords related to student topics (you can extend this)
    student_keywords = ["homework", "study", "math", "science", "history", "assignment", "school", "exam"]
    return any(keyword.lower() in query.lower() for keyword in student_keywords)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Check if the query is relevant to students
        if not is_student_query(student_input):
            return "This chatbot is only for educational questions. Please ask something related to your studies or school."

        # Modify the prompt to be more educational and relevant to the query
        prompt = f"You are a helpful assistant for students. Answer the following question in a way that helps a student understand the topic better:\n{student_input}"

        # Send the student's question to Groq's API and get a response
        response = llm.invoke(prompt)

        # Access the content directly from the response
        content = response.content  # Accessing content directly
        return content.strip()  # Return the content text

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any educational question, and I'll help you learn!"  # Student-focused description
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fdb7fc5262558d7349.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
